In [1]:
import pandas as pd
import numpy as np
import torchvision.models as models
import torch.nn as nn
import os
import torch
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

## Load images

In [2]:
img_path = './images/'
feature_path = './features/'

In [3]:
classes = []

In [4]:
classes = os.listdir(img_path)
classes = sorted(classes)

In [5]:
classes # Load classes for specific task

['antelope',
 'bat',
 'beaver',
 'blue+whale',
 'bobcat',
 'buffalo',
 'chihuahua',
 'chimpanzee',
 'collie',
 'cow',
 'dalmatian',
 'deer',
 'dolphin',
 'elephant',
 'fox',
 'german+shepherd',
 'giant+panda',
 'giraffe',
 'gorilla',
 'grizzly+bear',
 'hamster',
 'hippopotamus',
 'horse',
 'humpback+whale',
 'killer+whale',
 'leopard',
 'lion',
 'mole',
 'moose',
 'mouse',
 'otter',
 'ox',
 'persian+cat',
 'pig',
 'polar+bear',
 'rabbit',
 'raccoon',
 'rat',
 'rhinoceros',
 'seal',
 'sheep',
 'siamese+cat',
 'skunk',
 'spider+monkey',
 'squirrel',
 'tiger',
 'walrus',
 'weasel',
 'wolf',
 'zebra']

## Extract Res-Net features

In [6]:
def res_feature(img,pretrained_model='resnet50',feature_layer='avgpool',use_gpu=False):
    model = None
    # Check image is not None
    if img is None:
        prin("No input image!")
        return
    # Get model
    if pretrained_model == 'resnet50':
        model = models.resnet50(pretrained=True)
    elif pretrained_model == 'resnet101':
        model = models.resnet101(pretrained=True)
    elif pretrained_model == 'resnet152':
        model = models.resnet152(pretrained=True)
    else:
        print("None model input")
        return
    # Get feature layer
    for param in model.parameters():
        param.requires_grad = False
    conv = nn.Sequential(*list(model.children())[:-1])
    conv.eval()
    
    # pre-processing image
    scaler = transforms.Resize((224,224))
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    to_tensor = transforms.ToTensor()
    
    img_norm = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    if use_gpu:
        img_norm = img_norm.cuda()
        conv = conv.cuda()
    
    feature = conv(img_norm)   
    feature = np.array(feature.cpu().detach().numpy()).reshape((1,-1))
    return feature

In [7]:
model_type='resnet50'
img_path = './images/'
feature_path = './features/'+model_type+'/'
# domains = domains = ['clipart', 'infograph','painting','quickdraw','real','sketch']
#     classes = ['back_pack','bike','calculator','headphones','keyboard','laptop_computer','monitor','mouse','mug','projector']
use_gpu = torch.cuda.is_available()
# for domain in domains:
#     for s in ['train','test']:
#         if domain != 'clipart' or s != 'train':
#             continue
feats = np.array([])
labels = np.array([])
info = np.array([])
classes = os.listdir(img_path)
classes = sorted(classes)

i = 0

for c in range(len(classes)):
    label = c
    class_name = classes[c]
    print('Processing class: ', class_name, 'label: ', c)
    class_path = img_path + '/' + class_name
    files = os.listdir(class_path)
#     files = sorted(files)
    for f in range(len(files)):
        file_name = files[f]
        img_dir = class_path + '/' +file_name 
        try:
            img = Image.open(img_dir).convert('RGB')
            feature = res_feature(img,pretrained_model=model_type, use_gpu = use_gpu)
            feats = np.append(feats, feature)
            labels = np.append(labels, label)
            info = np.append(info, img_dir)
            
        except:
            print("!!!!!! Exception !!!!!!! Class: ", class_name, ' sample: ', img_dir)
feats = feats.reshape((-1,2048))
labels = labels.reshape((-1,1)).astype(int)
info = info.reshape((-1,1))
# save to csv file
print('Saving to ', model_type +'_feats.csv ...')
output_feats_file = feature_path + '/AwA2_' + model_type +'_feats.csv'
output_labels_file = feature_path + '/AwA2_' + model_type +'_labels.csv'
output_info_file = feature_path +'/AwA2_'+ model_type + '_info.csv'
res_feats = pd.DataFrame(feats)
res_labels = pd.DataFrame(labels)
res_info = pd.DataFrame(info)
res_feats.to_csv(output_feats_file,index=False,header=False)
res_labels.to_csv(output_labels_file,index=False,header=False)
res_info.to_csv(output_info_file,index=False,header=False)

Processing class:  antelope label:  0
Processing class:  bat label:  1
Processing class:  beaver label:  2
Processing class:  blue+whale label:  3
Processing class:  bobcat label:  4
Processing class:  buffalo label:  5
Processing class:  chihuahua label:  6
Processing class:  chimpanzee label:  7
Processing class:  collie label:  8
Processing class:  cow label:  9
Processing class:  dalmatian label:  10
Processing class:  deer label:  11
Processing class:  dolphin label:  12
Processing class:  elephant label:  13
Processing class:  fox label:  14
Processing class:  german+shepherd label:  15
Processing class:  giant+panda label:  16
Processing class:  giraffe label:  17
Processing class:  gorilla label:  18
Processing class:  grizzly+bear label:  19
Processing class:  hamster label:  20
Processing class:  hippopotamus label:  21
Processing class:  horse label:  22
Processing class:  humpback+whale label:  23
Processing class:  killer+whale label:  24
Processing class:  leopard label: 